In [11]:
import pandas as pd
import numpy as np
import import_ipynb
import spacy
from nltk.corpus import stopwords

In [12]:
import common_my_NLP

In [13]:
def make_frequency_dict(dictionary, neg_sentences, non_sentences):
    # idx = 0

    for sentence in neg_sentences:
        for word in sentence:
            
            if word not in dictionary:
                dictionary[word] = 0
                # idx += 1
            
            dictionary[word] += 1
    
    for sentence in non_sentences:
        for word in sentence:
            
            if word not in dictionary:
                dictionary[word] = 0
                # idx += 1
            
            dictionary[word] += 1

In [14]:
def make_idx_dict(dictionary, neg_sentences, non_sentences):
    idx = 0

    for sentence in neg_sentences:
        for word in sentence:
            
            if word not in dictionary:
                dictionary[word] = idx
                idx += 1
            
            # dictionary[word] += 1
    
    for sentence in non_sentences:
        for word in sentence:
            
            if word not in dictionary:
                dictionary[word] = idx
                idx += 1
            
            # dictionary[word] += 1

In [15]:
def rmv_frequency_dict(dictionary, low_bound):
    rmv_list = []
    for key, value in dictionary.items():
        if value <= low_bound:
            rmv_list.append(key)
    
    for word in rmv_list:
        dictionary.pop(word)

In [16]:
def idx_dictionary(dictionary):
    idx = 0

    for key, value in dictionary.items():
        dictionary[key] = idx
        idx += 1

In [17]:
def get_data():
    train_neg = pd.read_csv('data/train.negative.csv', quotechar=None, quoting=3, sep='\t', header=None)
    train_non = pd.read_csv('data/train.non-negative.csv', quotechar=None, quoting=3, sep='\t', header=None)

    test_neg = pd.read_csv('data/test.negative.csv', quotechar=None, quoting=3, sep='\t', header=None)
    test_non = pd.read_csv('data/test.non-negative.csv', quotechar=None, quoting=3, sep='\t', header=None)

    # train_neg.head()


    # train_neg[0] = train_neg[0].apply(lambda x: my_NLP.remove_html(x))
    # train_non[0] = train_non[0].apply(lambda x: my_NLP.remove_html(x))

    # test_neg[0] = test_neg[0].apply(lambda x: my_NLP.remove_html(x))
    # test_non[0] = test_non[0].apply(lambda x: my_NLP.remove_html(x))


    train_neg[0] = train_neg[0].apply(lambda x: common_my_NLP.remove_punctuation(x))
    train_non[0] = train_non[0].apply(lambda x: common_my_NLP.remove_punctuation(x))

    test_neg[0] = test_neg[0].apply(lambda x: common_my_NLP.remove_punctuation(x))
    test_non[0] = test_non[0].apply(lambda x: common_my_NLP.remove_punctuation(x))


    train_neg[0] = train_neg[0].apply(lambda x: x.lower())
    train_non[0] = train_non[0].apply(lambda x: x.lower())

    test_neg[0] = test_neg[0].apply(lambda x: x.lower())
    test_non[0] = test_non[0].apply(lambda x: x.lower())


    nlp = spacy.load('en_core_web_sm')

    tok_lem_sentence_train_neg = [[token.lemma_ for token in nlp(row[0].strip())] for index, row in train_neg.iterrows()]
    tok_lem_sentence_train_non = [[token.lemma_ for token in nlp(row[0].strip())] for index, row in train_non.iterrows()]

    tok_lem_sentence_test_neg = [[token.lemma_ for token in nlp(row[0].strip())] for index, row in test_neg.iterrows()]
    tok_lem_sentence_test_non = [[token.lemma_ for token in nlp(row[0].strip())] for index, row in test_non.iterrows()]

    # tok_lem_sentence_train_neg[0]


    stop_words = set(stopwords.words('english'))

    rmv_sw_sentence_train_neg = [ [word for word in sentence if not word in stop_words and word != ' '] for sentence in tok_lem_sentence_train_neg]
    rmv_sw_sentence_train_non = [ [word for word in sentence if not word in stop_words and word != ' '] for sentence in tok_lem_sentence_train_non]

    rmv_sw_sentence_test_neg = [ [word for word in sentence if not word in stop_words and word != ' '] for sentence in tok_lem_sentence_test_neg]
    rmv_sw_sentence_test_non = [ [word for word in sentence if not word in stop_words and word != ' '] for sentence in tok_lem_sentence_test_non]

    # rmv_sw_sentence_test_neg[0]

    dictionary = {}
    make_frequency_dict(dictionary, rmv_sw_sentence_train_neg, rmv_sw_sentence_train_non)
    # make_idx_dict(dictionary, rmv_sw_sentence_train_neg, rmv_sw_sentence_train_non)

    # dictionary


    rmv_frequency_dict(dictionary, 3)

    # dictionary


    idx_dictionary(dictionary)

    # dictionary


    train_neg_len = len(rmv_sw_sentence_train_neg)
    train_non_len = (len(rmv_sw_sentence_train_non)*2)-124

    test_neg_len = len(rmv_sw_sentence_test_neg)
    test_non_len = len(rmv_sw_sentence_test_non)

    train_row_len = (train_neg_len + train_non_len)
    test_row_len = (test_neg_len + test_non_len)
    col_len = len(dictionary)

    train_x = np.zeros((train_row_len, col_len))
    test_x = np.zeros((test_row_len, col_len))

    # train_x.shape


    idx = 0

    for sentence in rmv_sw_sentence_train_neg:
        for word in sentence:
            if word in dictionary:
                place = dictionary[word]
                train_x[idx][place] = 1
        idx += 1

    for sentence in rmv_sw_sentence_train_non:
        for word in sentence:
            if word in dictionary:
                place = dictionary[word]
                train_x[idx][place] = 1
        idx += 1

    for sentence in rmv_sw_sentence_train_non[:-124]:
        for word in sentence:
            if word in dictionary:
                place = dictionary[word]
                train_x[idx][place] = 1
        idx += 1


    idx = 0

    for sentence in rmv_sw_sentence_test_neg:
        for word in sentence:
            if word in dictionary:
                place = dictionary[word]
                test_x[idx][place] = 1
        idx += 1

    for sentence in rmv_sw_sentence_test_non:
        for word in sentence:
            if word in dictionary:
                place = dictionary[word]
                test_x[idx][place] = 1
        idx += 1


    train_y_non_set = np.ones((train_neg_len, 1))
    train_y_non_unset = np.zeros((train_neg_len, 1))
    train_y_neg_unset = np.zeros((train_non_len, 1))
    train_y_neg_set = np.ones((train_non_len, 1))

    tmp_train_y_non = np.concatenate((train_y_non_set, train_y_non_unset), axis=1)
    tmp_train_y_neg = np.concatenate((train_y_neg_unset, train_y_neg_set), axis=1)

    test_y_non_set = np.ones((test_neg_len, 1))
    test_y_non_unset = np.zeros((test_neg_len, 1))
    test_y_neg_unset = np.zeros((test_non_len, 1))
    test_y_neg_set = np.ones((test_non_len, 1))

    tmp_test_y_non = np.concatenate((test_y_non_set, test_y_non_unset), axis=1)
    tmp_test_y_neg = np.concatenate((test_y_neg_unset, test_y_neg_set), axis=1)

    # print(tmp_train_y_non.shape, tmp_train_y_neg.shape)
    # print(tmp_test_y_non.shape, tmp_test_y_neg.shape)


    train_y = np.concatenate((tmp_train_y_non, tmp_train_y_neg), axis=0)
    test_y = np.concatenate((tmp_test_y_non, tmp_test_y_neg), axis=0)

    # print(train_y.shape, test_y.shape)


    # print(train_x.shape, train_y.shape)
    # print(test_x.shape, test_y.shape)

    return train_x, train_y, test_x, test_y